In [152]:
import pandas as pd
import numpy as np

In [153]:
df = pd.read_csv('download.csv')

In [154]:
df['3day_tokenprice_mean'] = df['PRICE'].rolling(3).mean()
df['3day_liq_amount_mean'] = df['LIQUIDATION_AMOUNT_USD'].rolling(3).mean()
df['3day_tokenprice_std'] = df['PRICE'].rolling(3).std()
df['3day_liq_amount_std'] = df['LIQUIDATION_AMOUNT_USD'].rolling(3).std()





In [155]:
df["3day_price_shift"] = df['PRICE'] - df['PRICE'].shift(3)
df["3day_liq_amount_shift"] = df['LIQUIDATION_AMOUNT_USD'] - df['LIQUIDATION_AMOUNT_USD'].shift(3)


In [156]:
df['1std_significant_price_downtrend'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['3day_price_shift'] > (-df['3day_tokenprice_std']), 0)
df['1std_significant_liquidation_downtrend'] = pd.DataFrame(np.ones((len(df['3day_liq_amount_shift']) -1))).where(df['3day_price_shift'] < (df['3day_liq_amount_std']), 0)
df['5std_significant_price_downtrend'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['3day_price_shift'] > (-5 * df['3day_tokenprice_std']), 0)
df['5std_significant_liquidation_downtrend'] = pd.DataFrame(np.ones((len(df['3day_liq_amount_shift']) -1))).where(df['3day_price_shift'] < (5 * df['3day_liq_amount_std']), 0)
df['10std_significant_price_downtrend'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['3day_price_shift'] > (-10 * df['3day_tokenprice_std']), 0)
df['10std_significant_liquidation_downtrend'] = pd.DataFrame(np.ones((len(df['3day_liq_amount_shift']) -1))).where(df['3day_price_shift'] < (10 * df['3day_liq_amount_std']), 0)

In [157]:
df['1_correlated_significant_events'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['1std_significant_price_downtrend'] + df['1std_significant_liquidation_downtrend'] != 2, 0)
df['5_correlated_significant_events'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['5std_significant_price_downtrend'] + df['5std_significant_liquidation_downtrend'] != 2, 0)
df['10_correlated_significant_events'] = pd.DataFrame(np.ones((len(df['3day_price_shift']) -1))).where(df['10std_significant_price_downtrend'] + df['10std_significant_liquidation_downtrend'] != 2, 0)

In [165]:
print("Number of correlated events vs. individual events for 1 std:  \n")
print("Number of price and liquidation events: ", df['1_correlated_significant_events'].sum(), "\n")
print("Number of price events: ",  df['1std_significant_price_downtrend'].sum(), "\n" )
print("Number of liquidation events: ", df['1std_significant_liquidation_downtrend'].sum(), "\n") 
print("Number of correlated events vs. individual events for 5 std:  \n")
print("Number of price and liquidation events: ", df['5_correlated_significant_events'].sum(), "\n")
print("Number of price events: ",  df['5std_significant_price_downtrend'].sum(), "\n" )
print("Number of liquidation events: ", df['5std_significant_liquidation_downtrend'].sum(), "\n") 
print("Number of correlated events vs. individual events for 10 std:  \n")
print("Number of price and liquidation events: ", df['10_correlated_significant_events'].sum(), "\n")
print("Number of price events: ",  df['10std_significant_price_downtrend'].sum(), "\n" )
print("Number of liquidation events: ", df['10std_significant_liquidation_downtrend'].sum(), "\n") 

Number of correlated events vs. individual events for 1 std:  

Number of price and liquidation events:  104.0 

Number of price events:  157.0 

Number of liquidation events:  258.0 

Number of correlated events vs. individual events for 5 std:  

Number of price and liquidation events:  18.0 

Number of price events:  243.0 

Number of liquidation events:  258.0 

Number of correlated events vs. individual events for 10 std:  

Number of price and liquidation events:  12.0 

Number of price events:  249.0 

Number of liquidation events:  258.0 



In [160]:
def avg_recovery(price, price_shift, sig_events):
  recovery_sum = 0
  recovery_events = 0
  for i in range(len(price)):
    if sig_events[i] == 1:
      j = i
      if j == len(price):
          return recovery_sum / recovery_events
      while price[j] - price[i] < abs(price_shift[i]):
        j = j + 1
        if j == len(price):
          return recovery_sum / recovery_events
      recovery_sum += j - i
      recovery_events += 1
  return recovery_sum / recovery_events




  

In [162]:
avg_price_recovery_25 = avg_recovery(list(df['PRICE']), list(df['3day_price_shift']), list(df['1_correlated_significant_events']))
avg_price_recovery_50 = avg_recovery(list(df['PRICE']), list(df['3day_price_shift']), list(df['5_correlated_significant_events']))
avg_price_recovery_75 = avg_recovery(list(df['PRICE']), list(df['3day_price_shift']), list(df['10_correlated_significant_events']))
print("Average recovery time for a 25 std deviation correlated event", avg_price_recovery_25)
print("Average recovery time for a 50 std deviation correlated event", avg_price_recovery_50)
print("Average recovery time for a 75 std deviation correlated event", avg_price_recovery_75)

Average recovery time for a 25 std deviation correlated event 17.37
Average recovery time for a 50 std deviation correlated event 14.833333333333334
Average recovery time for a 75 std deviation correlated event 13.75
